In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
import re
import time
from random import randint

# spiegel-scraper

https://pypi.org/project/spiegel-scraper/

In [20]:
from datetime import date
import spiegel_scraper as spon
from datetime import datetime
from datetime import timedelta

### Seting up time period 

In [166]:
#for first loop ever
#start_date='2021-03-24'

In [171]:
start_date = datetime.today() - timedelta(days=7)
start_date = start_date.strftime("%Y-%m-%d")

In [172]:
start_date

'2021-04-02'

In [24]:
today = date.today()
today = today.strftime("%Y-%m-%d")

In [32]:
dates=pd.date_range(start_date,today,freq='D').strftime("%Y-%m-%d").tolist()
dates=[datetime.strptime(date, "%Y-%m-%d") for date in dates]
len(dates)

17

### Seting up API & Scraping data

In [33]:
url = []
headline = []
is_paid = []
date_published = []
channel = []

for date in dates:
    random_i = randint(5, 20)
    time.sleep(random_i)
    
    print()
    print(date)
    
    archive_entries_from_html = spon.archive.scrape_html(spon.archive.html_by_date(date))
    print()
    
    url += [archive_entries_from_html[i]["url"] for i in range(len(archive_entries_from_html))]
    headline += [archive_entries_from_html[i]["headline"] for i in range(len(archive_entries_from_html))]
    is_paid += [archive_entries_from_html[i]["is_paid"] for i in range(len(archive_entries_from_html))]
    date_published += [archive_entries_from_html[i]["date_published"] for i in range(len(archive_entries_from_html))]
    channel += [archive_entries_from_html[i]["channel"] for i in range(len(archive_entries_from_html))]
    print(len(channel))
    


2021-03-24 00:00:00

134

2021-03-25 00:00:00

265

2021-03-26 00:00:00

441

2021-03-27 00:00:00

511

2021-03-28 00:00:00

582

2021-03-29 00:00:00

691

2021-03-30 00:00:00

812

2021-03-31 00:00:00

936

2021-04-01 00:00:00

1042

2021-04-02 00:00:00

1144

2021-04-03 00:00:00

1212

2021-04-04 00:00:00

1281

2021-04-05 00:00:00

1343

2021-04-06 00:00:00

1455

2021-04-07 00:00:00

1567

2021-04-08 00:00:00

1673

2021-04-09 00:00:00

1746


In [141]:
spiegel = pd.DataFrame()

spiegel['url'] = [u for u in url]
spiegel['headline'] = [h for h in headline]
spiegel['is_paid'] = [p for p in is_paid]
spiegel['date_published'] = [date for date in date_published]
spiegel['channel'] = [c for c in channel]


In [142]:
spiegel.shape

(1746, 5)

### Data Preprocessing: Creating functions

In [143]:
import re
import string

In [144]:
def data_clean(x):
    x=str(x)
    return x[:10]

In [145]:
def cleaning_punctuation(x):
    x=str(x)
    #x=x.lower()
    remove = string.punctuation
    remove = remove.replace("-", "")
    remove = remove.replace("/", "")
    remove = remove.replace("&", "")# don't remove hyphens
    pattern = r"[{}]".format(remove)
    x=re.sub(pattern, "", x)
    return x

In [146]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

german_stop_words = stopwords.words('german')
vect = CountVectorizer(stop_words = german_stop_words)

In [147]:
def stop_word_removal(x):
    x=str(x)
    x=x.lower()
    token = x.split()
    return ' '.join([w for w in token if not w in german_stop_words])

In [148]:
import spacy

nlp = spacy.load('de_core_news_sm')

In [149]:
def lemmatizer(x):
    token = nlp(x)
    #x.lemma_
    return ' '.join([t.lemma_ for t in token])

### Data Preprocessing: Applying functions

In [150]:
spiegel["date_published"]=spiegel["date_published"].apply(lambda x: data_clean(x))
spiegel["headline"]=spiegel["headline"].apply(lambda x: cleaning_punctuation(x))
spiegel["headline"]=spiegel["headline"].apply(lambda x: re.sub('»', "", x))
spiegel["headline"]=spiegel["headline"].apply(lambda x: re.sub('«', "", x))
spiegel["headline"]=spiegel["headline"].apply(lambda x: re.sub('–', " ", x))
spiegel["headline"]=spiegel["headline"].apply(lambda x: re.sub('-', " ", x))
spiegel["original_headline"]=spiegel["headline"]
spiegel["headline"]=spiegel["headline"].apply(lambda x: stop_word_removal(x))
spiegel["headline"]=spiegel["headline"].apply(lambda x: lemmatizer(x))
spiegel["newspaper"]=spiegel["headline"].apply(lambda x: "Der Spiegel")

In [151]:
spiegel

,url,headline,is_paid,date_published,channel,original_headline,newspaper
0,https://www.spiegel.de/politik/ausland/us-auss...,gespräche brüssel blinken stellen eu eng zusam...,False,2021-03-24,Politik,Gespräche in Brüssel Blinken stellt EU enge Zu...,Der Spiegel
1,https://www.spiegel.de/sport/fussball/wm-quali...,wm qualifikation yilmaz lässt niederlanden cha...,False,2021-03-24,Sport,WM Qualifikation Yilmaz lässt Niederlanden kei...,Der Spiegel
2,https://www.spiegel.de/politik/ausland/usa-joe...,herausfordernde situation harris illegale migr...,False,2021-03-24,Politik,Herausfordernde Situation Harris soll sich um ...,Der Spiegel
3,https://www.spiegel.de/sport/fussball/u21-euro...,dfb auswahl siegen ungarn nmecha baku sorgen e...,False,2021-03-24,Sport,DFB Auswahl siegt gegen Ungarn Nmecha und Baku...,Der Spiegel
4,https://www.spiegel.de/sport/fussball/wm-quali...,wm qualifikation botschaft katar norwegens fuß...,False,2021-03-24,Sport,WM Qualifikation Botschaft an Katar Norwegen...,Der Spiegel
...,...,...,...,...,...,...,...
1741,https://www.spiegel.de/politik/ausland/nordirl...,erneute proteste nordirland randalierer werfen...,False,2021-04-09,Politik,Erneute Proteste in Nordirland Randalierer wer...,Der Spiegel
1742,https://www.spiegel.de/politik/ausland/nach-so...,sofagate italiens ministerpräsident draghi bez...,False,2021-04-09,Politik,Nach Sofagate Italiens Ministerpräsident Dragh...,Der Spiegel
1743,https://www.spiegel.de/kultur/musik/celeste-ne...,soul shootingstar celeste gehen kopf stimme be...,False,2021-04-09,Kultur,Soul Shootingstar Celeste Mir ging durch den K...,Der Spiegel
1744,https://www.spiegel.de/familie/corona-krise-wa...,windeln wäsche videocalls warum deutschland el...,False,2021-04-09,Familie,Windeln Wäsche Videocalls Warum Deutschland oh...,Der Spiegel


### Cleaning channel

In [152]:
channel_dict={}

for unique in list(spiegel["channel"].unique()):
    channel_dict[unique]=spiegel[spiegel["channel"]==unique]["channel"].count()

counts=spiegel["channel"].value_counts(normalize=True)
    
print("Before deleting channels: ", spiegel.shape)
spiegel = spiegel[~spiegel['channel'].isin(counts[counts < 0.01].index)]
print("After deleting channels: ", spiegel.shape)

Before deleting channels:  (1746, 7)
After deleting channels:  (1678, 7)


In [153]:
counts

Politik           0.307560
Panorama          0.180985
Wirtschaft        0.140321
Sport             0.107675
Kultur            0.073883
Wissenschaft      0.057847
Netzwelt          0.037801
Geschichte        0.012600
Gesundheit        0.010882
Psychologie       0.010882
Reise             0.010309
Mobilität         0.010309
Start             0.009737
Job & Karriere    0.008591
International     0.005727
Stil              0.005727
Familie           0.004582
Tests             0.001718
Partnerschaft     0.001145
Dein SPIEGEL      0.001145
Community         0.000573
Name: channel, dtype: float64

In [154]:
print("Before deleting channel 'Sport': ", spiegel.shape)

Before deleting channel 'Sport':  (1678, 7)


In [155]:
spiegel = spiegel[~spiegel['channel'].isin(["Sport"])]
print("After deleting channel 'Sport': ", spiegel.shape)

After deleting channel 'Sport':  (1490, 7)


In [173]:
start_date

'2021-04-02'

In [168]:
spiegel_old=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/vaccine_sentiment/german/automated_scraping/spiegel_"+start_date+".csv", index_col=0)
spiegel_old                       

,url,original_headline,headline,date_published,channel,newspaper,vaccine
9573,https://www.spiegel.de/panorama/italien-weitet...,coronavirus italien weitet rote zone auf das g...,coronavirus italien weiten rote zone ganze land,2020-03-09,Panorama,Der Spiegel,NaN
9574,https://www.spiegel.de/panorama/coronavirus-in...,coronavirus in brandenburg bis zu 2250 mensche...,coronavirus brandenburg 2250 menschen häuslich...,2020-03-09,Panorama,Der Spiegel,NaN
9575,https://www.spiegel.de/reise/fernweh/coronavir...,fotostrecke mekka venedig tokio wie das virus ...,fotostrecke mekka venedig tokio virus menschen...,2020-03-09,Reise,Der Spiegel,NaN
9576,https://www.spiegel.de/wissenschaft/mensch/dr-...,sprechstunde bei dr hackenbroch was sie über c...,sprechstunde dr hackenbroch covid19 wissen müssen,2020-03-09,Wissenschaft,Der Spiegel,NaN
9577,https://www.spiegel.de/wirtschaft/service/boer...,börsenbeben dax schließt mit größtem verlust s...,börsenbeben dax schließen groß verlust seit 09...,2020-03-09,Wirtschaft,Der Spiegel,NaN
...,...,...,...,...,...,...,...
22904,https://www.spiegel.de/politik/deutschland/cor...,streitthema gottesdienste seehofer kritisiert ...,streitthema gottesdienste seehofer kritisieren...,2021-03-24,Politik,Der Spiegel,NaN
22905,https://www.spiegel.de/wirtschaft/soziales/cor...,corona in verschiedenen einkommensgruppen ist ...,corona verschieden einkommensgruppen virus ega...,2021-03-24,Wirtschaft,Der Spiegel,NaN
22906,https://www.spiegel.de/reise/luxustourismus-tr...,luxustourismus trotz corona der tegernsee wird...,luxustourismus trotzen corona tegernsee neue sylt,2021-03-24,Reise,Der Spiegel,NaN
22907,https://www.spiegel.de/netzwelt/web/twitter-er...,24 zeichen erster tweet der welt bringt bei au...,24 zeichen erst tweet welt bringen auktion fas...,2021-03-24,Netzwelt,Der Spiegel,NaN


In [178]:
spiegel=pd.concat([spiegel_old, spiegel], ignore_index=True)
#spiegel

### Applying corona score

"corona_dictionary_german.csv" will be updated every week with the latest newspaper articels scrapped.

In [179]:
keywords=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/vaccine_sentiment/german/corona_dictionary_german.csv")

In [180]:
#keywords=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/vaccine_sentiment/german/corona_dictionary_german.csv")

corona_list=["corona","rki","wirkstoff","drosten","wieler","merkel","spahn","coronavirus","virus","lockdown","pandemie","epidemie","pcr-test","impfzentren","hausärzte","schnelltest","teststrategie","intensivbetten","covid","covid-19","impfung","impfstrategie","impfen","biontech","pfizer", "biontech/pfizer","moderna","johnson&johnson", "curevac","astrazeneca","oxford","janssen"]

def corona_related(x):
    temp_list=x.split(' ')

    score=0
    
    for word in temp_list:
        #print(word)
        if word in corona_list:
            score_corona=1
            #print(score)
            return score_corona
        
        elif word in list(keywords["keyword"]):
            score+=1
            #print(score)
        else:
            continue
            
    return score/(len(temp_list))

spiegel["corona_score"]=spiegel["headline"].apply(lambda x: corona_related(x))

In [181]:
spiegel=spiegel[spiegel["corona_score"]>0.5]

In [183]:
spiegel.drop_duplicates(inplace=True)

In [185]:
spiegel.reset_index(drop=True, inplace=True)

### Applying vaccine dictionary

In [201]:
vaccine=["geimpfte","impfzentrum","impfzentren","impfung","impfen","impfstrategie","wirkstoff","biontech","pfizer", "biontech/pfizer","moderna","johnson&johnson", "curevac","astrazeneca","astra-zeneca","oxford","janssen"]


def vaccine_mentioned(x):
    temp_list=x.split(' ')
    
    for word in temp_list:
        #print(word)
        if word in vaccine:
            return word

        else:
            return "n/a"
        
spiegel["vaccine"]=spiegel["headline"].apply(lambda x: vaccine_mentioned(x))

In [202]:
spiegel["vaccine"].value_counts()

n/a            13959
biontech          13
astrazeneca       11
impfen             4
impfung            3
wirkstoff          1
moderna            1
curevac            1
impfzentren        1
Name: vaccine, dtype: int64

In [195]:
today

'2021-04-09'

In [196]:
spiegel.to_csv("/Users/carolinvogt/Becoming_Data_Analyst/vaccine_sentiment/german/automated_scraping/spiegel_"+today+".csv")

print("Sucessfully saved")

Sucessfully saved


In [198]:
spiegel_vaccine=spiegel[spiegel["vaccine"]!="n/a"]
spiegel_vaccine

,url,original_headline,headline,date_published,channel,newspaper,vaccine,is_paid,corona_score
1796,https://www.spiegel.de/wissenschaft/medizin/co...,wirkstoff remdesivir möglicher durchbruch bei ...,wirkstoff remdesivir möglich durchbruch corona...,2020-04-17,Wissenschaft,Der Spiegel,wirkstoff,NaN,1.0
5338,https://www.spiegel.de/wissenschaft/medizin/co...,biontech mainzer unternehmen startet internati...,biontech mainzer unternehmen starten internati...,2020-07-28,Wissenschaft,Der Spiegel,biontech,NaN,1.0
6077,https://www.spiegel.de/wirtschaft/unternehmen/...,curevac impfstoffhersteller ist jetzt doppelt ...,curevac impfstoffhersteller doppeln soviel wer...,2020-08-20,Wirtschaft,Der Spiegel,curevac,NaN,1.0
6691,https://www.spiegel.de/wissenschaft/medizin/co...,astrazeneca stoppt studien mit coronaimpfstoff...,astrazeneca stoppen studien coronaimpfstoff rü...,2020-09-09,Wissenschaft,Der Spiegel,astrazeneca,NaN,1.0
7549,https://www.spiegel.de/wissenschaft/medizin/co...,biontech euarzneibehörde startet begutachtung ...,biontech euarzneibehörde starten begutachtung ...,2020-10-06,Wissenschaft,Der Spiegel,biontech,NaN,1.0
8796,https://www.spiegel.de/politik/ausland/coronav...,biontech und pfizer euvertrag für coronaimpfst...,biontech pfizer euvertrag coronaimpfstoff aush...,2020-11-10,Politik,Der Spiegel,biontech,NaN,1.0
8821,https://www.spiegel.de/wissenschaft/medizin/co...,biontech und co wo die welt im impfstoffrennen...,biontech co welt impfstoffrennen stehen,2020-11-11,Wissenschaft,Der Spiegel,biontech,NaN,1.0
9187,https://www.spiegel.de/wissenschaft/medizin/co...,astrazeneca und uni oxford der impfstoff für p...,astrazeneca uni oxford impfstoff pragmatiker,2020-11-23,Wissenschaft,Der Spiegel,astrazeneca,NaN,1.0
9921,https://www.spiegel.de/politik/deutschland/cor...,mit biontech/pfizervakzine impfungen in deutsc...,biontech / pfizervakzine impfungen deutschland...,2020-12-16,Politik,Der Spiegel,biontech,NaN,1.0
9951,https://www.spiegel.de/wissenschaft/medizin/eu...,moderna eubehörde beschleunigt zulassung von u...,moderna eubehörde beschleunigen zulassung usim...,2020-12-17,Wissenschaft,Der Spiegel,moderna,NaN,1.0
